In [1]:
import pathlib

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tifffile as tf  # write tiff files
from PIL import Image  # read tiff files
from toml import load
from tqdm import tqdm  # progress bar

In [2]:
CELL_TYPE = "PBMC"

In [3]:
path = pathlib.Path(
    "../../../8.cytopick_analysis/results/PBMC/single_cell_predictions.parquet"
).resolve(strict=True)

# read in the data
df = pd.read_parquet(path)
# show all columns
pd.set_option("display.max_columns", None)
df.head()

,apoptosis_prob,control_prob,pyroptosis_prob,label_true,label_pred,data_split,shuffle,class_name,index,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_Site,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,Metadata_inducer1,Metadata_inducer1_concentration,Metadata_inducer1_concentration_unit,Metadata_inducer2,Metadata_inducer2_concentration,Metadata_inducer2_concentration_unit,Metadata_ImageNumber,Metadata_Plate,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_Nuclei_Number_Object_Number,Metadata_Treatment,Metadata_Dose,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cytoplasm_AreaShape_BoundingBoxMaximum_X,Metadata_Cytoplasm_AreaShape_BoundingBoxMaximum_Y,Metadata_Cytoplasm_AreaShape_BoundingBoxMinimum_X,Metadata_Cytoplasm_AreaShape_BoundingBoxMinimum_Y,labels,correct,image_path,image_crop_path
0,0.053009,0.227769,0.719221,2,2,train,True,pyroptosis,3599724.0,PBMC,J08,37243.0,3.0,6.0,DMSO,0.025,%,LPS,100.000,µg_per_ml,None,nan,None,99.0,*70117_20230210MM1_Gasdermin514_CP_BC430856,1074.0,1074.0,1075.0,1075.0,LPS,100.000_µg_per_ml,333.556164,701.619178,345.0,713.0,323.0,690.0,pyroptosis,True,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...
1,0.069751,0.241888,0.688361,2,2,train,True,pyroptosis,724498.0,PBMC,C11,45842.0,10.0,6.0,DMSO,0.025,%,LPS,100.000,µg_per_ml,Nigericin,3.000,µM,330.0,*70117_20230210MM1_Gasdermin514_CP_BC430856,2740.0,2740.0,2742.0,2742.0,LPS_Nigericin,100.000_µg_per_ml_3.000_µM,613.582996,1752.874494,623.0,1762.0,605.0,1745.0,pyroptosis,True,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...
2,0.081726,0.323375,0.594899,2,2,train,True,pyroptosis,579466.0,PBMC,B10,43892.0,5.0,6.0,DMSO,0.025,%,LPS,100.000,µg_per_ml,Nigericin,1.000,µM,133.0,*70117_20230210MM1_Gasdermin514_CP_BC430856,540.0,540.0,541.0,541.0,LPS_Nigericin,100.000_µg_per_ml_1.000_µM,1477.613687,413.838852,1491.0,427.0,1463.0,403.0,pyroptosis,True,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...
3,0.092082,0.327625,0.580293,2,2,train,True,pyroptosis,3701103.0,PBMC,J09,39448.0,13.0,6.0,DMSO,0.025,%,LPS,100.000,µg_per_ml,None,nan,None,125.0,*70117_20230210MM1_Gasdermin514_CP_BC430856,289.0,289.0,289.0,289.0,LPS,100.000_µg_per_ml,1981.429885,177.239080,1994.0,191.0,1971.0,166.0,pyroptosis,True,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...
4,0.075811,0.231210,0.692979,2,2,testing,True,pyroptosis,3689653.0,PBMC,J09,39448.0,8.0,6.0,DMSO,0.025,%,LPS,100.000,µg_per_ml,None,nan,None,120.0,*70117_20230210MM1_Gasdermin514_CP_BC430856,2204.0,2204.0,2210.0,2210.0,LPS,100.000_µg_per_ml,631.633333,1406.147619,660.0,1434.0,616.0,1392.0,pyroptosis,True,/home/lippincm/Documents/ML/Interstellar_Analy...,/home/lippincm/Documents/ML/Interstellar_Analy...


In [4]:
# loop through each image and assign a scale bar
for image_path in df["image_crop_path"]:
    print(image_path)
    # open the image
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    # image to array
    image_array = np.array(image)
    resolution = 0.1550  # pixels per micrometer
    # get the image size
    image_size = image_array.shape
    if image_array.shape[1] <= 100:
        scale_bar_length = 5  # um
        scale_bar_height = 1  # pixels
        padding = 3  # pixels
    elif image_array.shape[1] > 100:
        scale_bar_length = 100
        scale_bar_height = 10
        padding = 10
    # get the bottom right most corner based on scale 1 % of pixels
    scale_bar_x = image_size[1] - (scale_bar_length / resolution) - padding - padding
    scale_bar_y = image_size[0] - (scale_bar_height) - padding
    print(scale_bar_x, scale_bar_y)
    # draw the scale bar
    new = cv2.rectangle(
        image_array,
        (int(scale_bar_x), int(scale_bar_y)),
        (image_size[1] - padding, image_size[0] - padding),
        (255, 255, 255),
        -1,
    )
    # save the image with the scale bar via cv2
    cv2.imwrite(image_path, new)
#

/home/lippincm/Documents/ML/Interstellar_Analysis/8.cytopick_analysis/figures/PBMC/pyroptosis_shuffled_train_df_im1_im3_im4_composite_image_crop_cell_0.png
61.741935483870975 96
/home/lippincm/Documents/ML/Interstellar_Analysis/8.cytopick_analysis/figures/PBMC/pyroptosis_shuffled_train_df_im1_im3_im4_composite_image_crop_cell_1.png
61.741935483870975 96
/home/lippincm/Documents/ML/Interstellar_Analysis/8.cytopick_analysis/figures/PBMC/pyroptosis_shuffled_train_df_im1_im3_im4_composite_image_crop_cell_3.png
61.741935483870975 96
/home/lippincm/Documents/ML/Interstellar_Analysis/8.cytopick_analysis/figures/PBMC/pyroptosis_shuffled_train_df_im1_im3_im4_composite_image_crop_cell_4.png
61.741935483870975 96
/home/lippincm/Documents/ML/Interstellar_Analysis/8.cytopick_analysis/figures/PBMC/pyroptosis_shuffled_test_df_im1_im3_im4_composite_image_crop_cell_0.png
61.741935483870975 96
/home/lippincm/Documents/ML/Interstellar_Analysis/8.cytopick_analysis/figures/PBMC/pyroptosis_shuffled_test_df_